**TF-idf**

In [3]:
# import the libraries
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk
from nltk.corpus import stopwords

In [4]:
# Load data file
with open('/content/sample_repository.json') as in_file:
    data = json.load(in_file)

titles = [item[0] for item in data['data']]
documents = [item[1] for item in data['data']]

print(f'{len(documents)} documents')
# print the documnets
for idx in range(10):
    print(idx, " \t ", titles[idx], " : \t", documents[idx][:100])

32 documents
0  	  Pomegranate Bhagwa  : 	 Fresh Pomegranate from Anushka Avni International Bhagwa is a premium Pomegranate variety from India
1  	  Pomegranate Arakta  : 	 Fresh Pomegranate Arakta from Anushka Avni International This Pomegranate are bigger in size, sweet 
2  	  About Us  : 	 About Us Anushka Avni International (AAI) takes pleasure in presenting itself as one of the renowned
3  	  Contact Us  : 	 About Us Anushka Avni International (AAI) takes pleasure in presenting itself as one of the renowned
4  	  White Onions  : 	 White Onions from Anushka Avni International Fresh White Onion, which is widely acclaimed for its he
5  	  Video Gallery  : 	 Anushka Avni International (AAI) takes pleasure in presenting itself as one of the renowned Supplier
6  	  Food classes  : 	 To a botanist, a fruit is an entity that develops from the fertilized ovary of a flower. This means 
7  	  Nutrition  : 	 Nutrition is the biochemical and physiological process by which an organism uses int

In [5]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
search_terms = 'fruits'
# search_terms = 'vegetables'
# search_terms = 'healthy foods in Canada'

vectorizer = TfidfVectorizer(stop_words=stopwords)
vectors = vectorizer.fit_transform([search_terms] + documents)

# Calculate the word frequency, and calculate the cosine similarity of the search terms to the documents
cosine_similarities = linear_kernel(vectors[0:1], vectors).flatten()
document_scores = [item.item() for item in cosine_similarities[1:]]  # convert back to native Python dtypes

# Print the top 5 results and their titles
score_titles = [(score, title) for score, title in zip(document_scores, titles)]

for score, title in (sorted(score_titles, reverse=True, key=lambda x: x[0])[:5]):
    print(f'{score:0.3f} \t {title}')

0.165 	 Food classes
0.073 	 Canada's Food Guide
0.000 	 Pomegranate Bhagwa
0.000 	 Pomegranate Arakta
0.000 	 About Us


In [7]:
# search_terms = 'fruits'
search_terms = 'vegetables'
# search_terms = 'healthy foods in Canada'

vectorizer = TfidfVectorizer(stop_words=stopwords)
vectors = vectorizer.fit_transform([search_terms] + documents)

# Calculate the word frequency, and calculate the cosine similarity of the search terms to the documents
cosine_similarities = linear_kernel(vectors[0:1], vectors).flatten()
document_scores = [item.item() for item in cosine_similarities[1:]]  # convert back to native Python dtypes

# Print the top 5 results and their titles
score_titles = [(score, title) for score, title in zip(document_scores, titles)]

for score, title in (sorted(score_titles, reverse=True, key=lambda x: x[0])[:5]):
    print(f'{score:0.3f} \t {title}')

0.080 	 Canada's Food Guide
0.000 	 Pomegranate Bhagwa
0.000 	 Pomegranate Arakta
0.000 	 About Us
0.000 	 Contact Us


In [8]:
# search_terms = 'fruits'
# search_terms = 'vegetables'
search_terms = 'healthy foods in Canada'

vectorizer = TfidfVectorizer(stop_words=stopwords)
vectors = vectorizer.fit_transform([search_terms] + documents)

# Calculate the word frequency, and calculate the cosine similarity of the search terms to the documents
cosine_similarities = linear_kernel(vectors[0:1], vectors).flatten()
document_scores = [item.item() for item in cosine_similarities[1:]]  # convert back to native Python dtypes

# Print the top 5 results and their titles
score_titles = [(score, title) for score, title in zip(document_scores, titles)]

for score, title in (sorted(score_titles, reverse=True, key=lambda x: x[0])[:5]):
    print(f'{score:0.3f} \t {title}')

0.598 	 Canada's Food Guide
0.296 	 Diet
0.104 	 Canadian Industry Statistics
0.085 	 Major Market
0.073 	 Ford Bronco 


**GloVe**

In [9]:
import gensim
print(gensim.__version__)

4.0.1


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [10]:
# pip install gensim==4.0.1

In [11]:
#import libraries
import json
import logging
from re import sub
from multiprocessing import cpu_count
import numpy as np
import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

In [12]:
# Support functions for pre-processing and calculation
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

In [14]:
query_string = 'fruits'

# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in documents]
query = preprocess(query_string)

# Download and/or load the GloVe word vector embeddings

if 'glove' not in locals():
    glove = api.load("glove-wiki-gigaword-50")
    
similarity_index = WordEmbeddingSimilarityIndex(glove)

# Build the term dictionary, TF-idf model
# The search query must be in the dictionary as well, in case the terms do not overlap with the documents
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

[==================================================] 100.0% 66.0/66.0MB downloaded


100%|██████████| 568/568 [00:16<00:00, 35.26it/s]
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:359: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:359: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


In [15]:
# Output the similarity scores for top 10 documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes[:10]:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {titles[idx]}')

6 	 0.884 	 Food classes
31 	 0.844 	 fruit serving bowl
29 	 0.844 	 List of fruit dishes
0 	 0.809 	 Pomegranate Bhagwa
10 	 0.761 	 Canada's Food Guide
1 	 0.725 	 Pomegranate Arakta
25 	 0.693 	 history of botany
23 	 0.630 	 Grapes Flame / Red Seedless
16 	 0.566 	 About Us
8 	 0.545 	 Nutrients


In [16]:
query_string = 'vegetables'

# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in documents]
query = preprocess(query_string)

# Download and/or load the GloVe word vector embeddings

if 'glove' not in locals():
    glove = api.load("glove-wiki-gigaword-50")
    
similarity_index = WordEmbeddingSimilarityIndex(glove)

# Build the term dictionary, TF-idf model
# The search query must be in the dictionary as well, in case the terms do not overlap with the documents
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix. 
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

100%|██████████| 568/568 [00:16<00:00, 34.65it/s]
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:359: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:359: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


In [17]:
# Output the similarity scores for top 10 documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes[:10]:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {titles[idx]}')


6 	 0.896 	 Food classes
10 	 0.810 	 Canada's Food Guide
29 	 0.760 	 List of fruit dishes
17 	 0.750 	 Small Onions
0 	 0.726 	 Pomegranate Bhagwa
4 	 0.708 	 White Onions
9 	 0.679 	 Diet
1 	 0.668 	 Pomegranate Arakta
31 	 0.659 	 fruit serving bowl
13 	 0.657 	 Red Onions


In [18]:
query_string = 'healthy foods in Canada'

# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in documents]
query = preprocess(query_string)

# Download and/or load the GloVe word vector embeddings

if 'glove' not in locals():
    glove = api.load("glove-wiki-gigaword-50")
    
similarity_index = WordEmbeddingSimilarityIndex(glove)

# Build the term dictionary, TF-idf model
# The search query must be in the dictionary as well, in case the terms do not overlap with the documents
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix. 
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

100%|██████████| 568/568 [00:16<00:00, 34.60it/s]
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:359: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:359: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


In [19]:
# Output the similarity scores for top 10 documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes[:10]:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {titles[idx]}')

10 	 0.939 	 Canada's Food Guide
9 	 0.684 	 Diet
31 	 0.589 	 fruit serving bowl
16 	 0.588 	 About Us
0 	 0.527 	 Pomegranate Bhagwa
19 	 0.526 	 Downloads
5 	 0.526 	 Video Gallery
2 	 0.526 	 About Us
3 	 0.526 	 Contact Us
20 	 0.509 	 Welcome to Anushka Avni International (AAI)


**BERT**

In [20]:
pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.4 MB/s 
     |████████████████████████████████| 3.1 MB 18.0 MB/s 
     |████████████████████████████████| 3.3 MB 39.7 MB/s 
     |████████████████████████████████| 1.2 MB 61.1 MB/s 
     |████████████████████████████████| 59 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 53.6 MB/s 
     |████████████████████████████████| 596 kB 49.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=91fb3f797647a417c36a30722cb07c56e1255d2aa0b709ed1b408076227a6093
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [21]:
#import libraries
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [22]:
# List of models optimized for semantic textual similarity can be found at:
# https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/edit#gid=0
model = SentenceTransformer('stsb-roberta-large')

Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [23]:
query = "fruits"
# encode sentences to get their embeddings
embedding1 = model.encode(documents, convert_to_tensor=True)
embedding2 = model.encode(query, convert_to_tensor=True)
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

In [24]:
# Print the result
for i in range(len(documents)):
  print("Documnet:", [i], documents[i])
  print("Query:", query)
  print("Similarity Score:", cosine_scores[i].item())
  print()

Documnet: [0] Fresh Pomegranate from Anushka Avni International Bhagwa is a premium Pomegranate variety from India. The Deep Red arils & the pleasing Red but rugged skin enhances the appearance whilst promoting shelf life of the fruit. Bhagwa is widely known for its soft seed, Dark red color and extremely delicious. Packaging: Net weight of box 2.5kg, 3.00kg, 3.5kg. Details: Minimum Weight 180gm, maximum weight 400gm Color of arils: Dark Cherry red. Taste: Sweet Fruit count / carton (3.50 kg net wt.) 9 Numbers packed per carton: 350-400gms
Query: fruits
Similarity Score: 0.3557474613189697

Documnet: [1] Fresh Pomegranate Arakta from Anushka Avni International This Pomegranate are bigger in size, sweet with soft seeds, bold red arils. It also possess glossy, attractive, dark red skin. Packaging: Net weight of box 2.5kg, 3.00kg, 3.5kg. Details: Minimum Weight 180gm, maximum weight 400gm Taste: Sweet Fruit count / carton (3.50 kg net wt.) 9 Numbers packed per carton: 350-400gms 10 Number

In [25]:
query = "vegetables"
# encode sentences to get their embeddings
embedding1 = model.encode(documents, convert_to_tensor=True)
embedding2 = model.encode(query, convert_to_tensor=True)
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

In [26]:
# Print the result
for i in range(len(documents)):
  print("Documnet:", [i], documents[i])
  print("Query:", query)
  print("Similarity Score:", cosine_scores[i].item())
  print()

Documnet: [0] Fresh Pomegranate from Anushka Avni International Bhagwa is a premium Pomegranate variety from India. The Deep Red arils & the pleasing Red but rugged skin enhances the appearance whilst promoting shelf life of the fruit. Bhagwa is widely known for its soft seed, Dark red color and extremely delicious. Packaging: Net weight of box 2.5kg, 3.00kg, 3.5kg. Details: Minimum Weight 180gm, maximum weight 400gm Color of arils: Dark Cherry red. Taste: Sweet Fruit count / carton (3.50 kg net wt.) 9 Numbers packed per carton: 350-400gms
Query: vegetables
Similarity Score: 0.2927153408527374

Documnet: [1] Fresh Pomegranate Arakta from Anushka Avni International This Pomegranate are bigger in size, sweet with soft seeds, bold red arils. It also possess glossy, attractive, dark red skin. Packaging: Net weight of box 2.5kg, 3.00kg, 3.5kg. Details: Minimum Weight 180gm, maximum weight 400gm Taste: Sweet Fruit count / carton (3.50 kg net wt.) 9 Numbers packed per carton: 350-400gms 10 Nu

In [27]:
query = "healthy foods in Canada"
# encode sentences to get their embeddings
embedding1 = model.encode(documents, convert_to_tensor=True)
embedding2 = model.encode(query, convert_to_tensor=True)
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

In [28]:
# Print the result
for i in range(len(documents)):
  print("Documnet:", [i], documents[i])
  print("Query:", query)
  print("Similarity Score:", cosine_scores[i].item())
  print()

Documnet: [0] Fresh Pomegranate from Anushka Avni International Bhagwa is a premium Pomegranate variety from India. The Deep Red arils & the pleasing Red but rugged skin enhances the appearance whilst promoting shelf life of the fruit. Bhagwa is widely known for its soft seed, Dark red color and extremely delicious. Packaging: Net weight of box 2.5kg, 3.00kg, 3.5kg. Details: Minimum Weight 180gm, maximum weight 400gm Color of arils: Dark Cherry red. Taste: Sweet Fruit count / carton (3.50 kg net wt.) 9 Numbers packed per carton: 350-400gms
Query: healthy foods in Canada
Similarity Score: 0.3625621795654297

Documnet: [1] Fresh Pomegranate Arakta from Anushka Avni International This Pomegranate are bigger in size, sweet with soft seeds, bold red arils. It also possess glossy, attractive, dark red skin. Packaging: Net weight of box 2.5kg, 3.00kg, 3.5kg. Details: Minimum Weight 180gm, maximum weight 400gm Taste: Sweet Fruit count / carton (3.50 kg net wt.) 9 Numbers packed per carton: 350